# Prereq

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import pandas as pd
import random
import nltk

In [2]:
!pip install datasets

     |████████████████████████████████| 290 kB 5.4 MB/s 
     |████████████████████████████████| 132 kB 48.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.0 MB/s 
     |████████████████████████████████| 59 kB 5.6 MB/s 
     |████████████████████████████████| 243 kB 46.4 MB/s 
     |████████████████████████████████| 160 kB 48.1 MB/s 
     |████████████████████████████████| 192 kB 44.8 MB/s 
     |████████████████████████████████| 271 kB 46.9 MB/s 


In [3]:
if(torch.cuda.is_available()):
    device = torch.device("cuda")
    print('Using GPU')
    print('GPU count:', torch.cuda.device_count())
    print('GPU device:', torch.cuda.get_device_name(0))
else:
    print('Using CPU')
    device = torch.device("cpu")

Using GPU
GPU count: 1
GPU device: Tesla K80


# Getting the dataset ready

## Download dataset

In [4]:
from datasets import load_dataset

dataset = load_dataset("dbpedia_14")

Downloading:   0%|          | 0.00/2.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.3M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset d_bpedia14 downloaded and prepared to /root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
train_total = dataset['train']
# test_total = dataset['test']

train_data_x = []
train_data_y = []

for i in random.sample(range(train_total.shape[0]), 20000):
    tmp = list(train_total[i].values())
    train_data_x.append(tmp[2])
    train_data_y.append(tmp[0])

In [6]:
df = pd.DataFrame({"sentence": train_data_x, "class": train_data_y})
df.sample(10)

,sentence,class
18678,Kamal Amin Awad (Arabic: كمال أمين عوض‎) (bor...,2
18427,The University of Seoul (UOS) (Korean: 서울시립대학...,1
232,Carl Enckell (7 June 1876 Saint Petersburg – ...,4
5572,Kingdom of Desire is the eighth studio album ...,11
5721,The Florida Institute of Technology (commonly...,1
13978,The fifth US Coast Guard cutter named Mohawk ...,5
10320,Harry Walter Perkowski (born September 6 1922...,3
2270,Other Roads is an album by Boz Scaggs release...,11
11077,Girl Friends (Russian: Подруги) is a 1936 Sov...,12
5800,Labu dan Labi (Labu and Labi) is a 1962 Malay...,12


## Preprocessing dataset

In [7]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

import gensim.downloader as api
from gensim.models.word2vec import Word2Vec

text_embedder = api.load("glove-twitter-25")

# for gensim info
# https://github.com/kavgan/nlp-in-practice/blob/master/pre-trained-embeddings/Pre-trained%20embeddings.ipynb

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[==================================================] 100.0% 104.8/104.8MB downloaded


In [8]:
import re
# regex = r"[^a-zA-Z0-9\-:;,.!?() ]+"
regex = r"[^a-zA-Z0-9,. ]+"
max_word_count = 100

def cleanSentence(sen):
    sen = sen.replace(".", ". ")
    sen = sen.replace(",", ", ")
    result = re.sub(regex, " ", sen).lower()
    result = word_tokenize(result)
    tmp = []
    for word in result:
        if(word in text_embedder and len(tmp) < max_word_count):
            tmp.append(text_embedder.vocab[word].index)
    tmp = tmp + [-1] * (max_word_count - len(tmp))
    return tmp

In [9]:
def classVector(label, count):
    tmp = [0] * count
    tmp[label] = 1
    return tmp

In [10]:
print(train_data_x[16737])
print(cleanSentence(train_data_x[16737]))
print(classVector(train_data_y[16737], 14))

 Millikin University is an American co-educational comprehensive private four-year university with traditional undergraduate programs in arts and sciences business fine arts and professional studies as well as non-traditional adult degree-completion programs (PACE) and graduate programs in business administration and nursing. Millikin's campus is in Decatur Illinois and serves approximately 2400 students with a student/faculty ratio of 12 to 1 and an average class size of 23 students.
[941992, 4490, 32, 172, 2033, 2384, 32883, 50531, 4734, 2866, 356, 4490, 58, 18468, 145241, 17913, 35, 9537, 26, 46688, 1249, 1121, 9537, 26, 6502, 11218, 124, 296, 124, 682, 18468, 6800, 9310, 62141, 17913, 11464, 26, 8133, 17913, 35, 1249, 21529, 26, 17896, 1, 941992, 137, 6399, 32, 35, 76445, 18895, 26, 32481, 32241, 4013, 58, 11, 3823, 42795, 34298, 39, 16, 26, 172, 5443, 726, 2361, 39, 4013, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

In [11]:
sentences = []
for i in range(len(train_data_x)):
    sentences.append(cleanSentence(train_data_x[i]))

labels = []
labelcount = max(train_data_y) + 1
for i in range(len(train_data_y)):
    labels.append(classVector(train_data_y[i], labelcount))

sentences = torch.tensor(sentences)
labels = torch.tensor(labels)

In [12]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pack_sequence

def collate_fn(data):
    feat, label = zip(*data)
    batch_size = len(feat)
    tok_size = len(feat[0])
    vec_size = 25
    features = []
    for index, fex in enumerate(feat):
        tmp = []
        for wordind in range(tok_size):
            if(fex[wordind] == -1):
                break
            else:
                tmp.append(torch.tensor(text_embedder[text_embedder.index2word[fex[wordind]]]))
        features.append(torch.stack(tmp))
    label = torch.stack(label)
    return pack_sequence(features, enforce_sorted=False), label

batch_size = 32

In [13]:
train_dataset = TensorDataset(sentences, labels)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, collate_fn=collate_fn)

# Model

In [17]:
class LSTMClassifier(nn.Module):
    def __init__(self, feature_dim, hidden_dim, num_layers, num_classes, batch_size):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_dim
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.lstm = nn.LSTM(input_size=feature_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device))
        c_0 = Variable(torch.zeros(self.num_layers, self.batch_size, self.hidden_size).to(device))
        output, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        h_n = h_n.view(-1, self.hidden_size)
        out = self.fc(h_n)
        out = self.softmax(out)
        return out

# Training



In [18]:
n_epochs = 10
lr = 0.01
feature_dim = 25
hidden_dim = 50
num_layers = 1

lstm = LSTMClassifier(feature_dim=feature_dim, hidden_dim=hidden_dim, num_layers=1, num_classes=labelcount, batch_size=batch_size)
lstm.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=lr)

In [19]:
import time

for epoch in range(n_epochs):
    avg_loss = 0
    st = time.time()
    for step, batch in enumerate(train_dataloader):
        x, y = batch
        x = x.to(device)
        y = y.float().to(device)
        y_pred = lstm(x)
        optimizer.zero_grad()
        loss = criterion(y_pred, y)
        avg_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_loss /= len(train_dataloader)
    if(epoch % 1 == 0):
      print(f"Epoch {epoch}: loss = {avg_loss:.6f}, time taken = {time.time() - st:.3f}")

Epoch 0: loss = 2.393912, time taken = 35.735
Epoch 1: loss = 2.054529, time taken = 35.652
Epoch 2: loss = 1.921264, time taken = 35.466
Epoch 3: loss = 1.837187, time taken = 35.783
Epoch 4: loss = 1.825772, time taken = 35.684
Epoch 5: loss = 1.824035, time taken = 35.407
Epoch 6: loss = 1.819137, time taken = 35.371
Epoch 7: loss = 1.812408, time taken = 35.497
Epoch 8: loss = 1.807745, time taken = 35.998
Epoch 9: loss = 1.804931, time taken = 35.719


# Testing

In [20]:
test_total = dataset['test']

test_data_x = []
test_data_y = []

for i in random.sample(range(test_total.shape[0]), 1000):
    tmp = list(test_total[i].values())
    test_data_x.append(tmp[2])
    test_data_y.append(tmp[0])

sentences = []
for i in range(len(test_data_x)):
    sentences.append(cleanSentence(test_data_x[i]))

sentences = torch.tensor(sentences)
labels = torch.tensor(test_data_y)

test_dataset = TensorDataset(sentences, labels)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

In [21]:
with torch.no_grad():
    count = 0
    for step, batch in enumerate(test_dataloader):
        x, y = batch
        x = x.to(device)
        y = y.to(device)
        y_pred = lstm(x)
        y_label = torch.argmax(y_pred, dim=1)
        count += torch.sum(y == y_label).item()
    print(f'Accuracy of the LSTM model: {count * 100 / sentences.shape[0]:.2f} %')

Accuracy of the LSTM model: 93.90 %
